In [1]:
import scipy.io as sio
import numpy as np
import os
import fileinput
import plotly.graph_objs as go
import sklearn
from scipy.cluster.vq import whiten
from sklearn import cluster
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn import decomposition
import matplotlib.pyplot as plt
import plotly.plotly as py
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles
%matplotlib inline

In [2]:
# Open all the folders and load the data for the features data FOLD_3 

dic = {}
number_of_files = 0
name_of_files = list()
path = r'C:/Users/pinouche/Documents/ImperialData/FOLD_3'

for filename in os.listdir(path):
    dic[filename] = sio.loadmat(path+'/'+filename)
    number_of_files = number_of_files + 1
    name_of_files.append(filename)
    
print([number_of_files,name_of_files])

[40, ['20120213_seq1_01_01.mat', '20120213_seq1_01_02.mat', '20120213_seq2_01_01.mat', '20120213_seq2_01_02.mat', '20120326_seq3_01_01.mat', '20120326_seq3_01_02.mat', '20120423_seq10_01_01.mat', '20120423_seq10_01_02.mat', '20120423_seq11_01_01.mat', '20120423_seq11_01_02.mat', '20120423_seq1_01_01.mat', '20120423_seq1_01_02.mat', '20120423_seq2_01_01.mat', '20120423_seq2_01_02.mat', '20120423_seq3_01_01.mat', '20120423_seq3_01_02.mat', '20120423_seq4_01_01.mat', '20120423_seq4_01_02.mat', '20120423_seq6_01_01.mat', '20120423_seq6_01_02.mat', '20120423_seq8_01_01.mat', '20120423_seq8_01_02.mat', '20120423_seq9_01_01.mat', '20120423_seq9_01_02.mat', '20120430_seq10_01_01.mat', '20120430_seq10_01_02.mat', '20120430_seq11_01_01.mat', '20120430_seq11_01_02.mat', '20120430_seq12_01_01.mat', '20120430_seq12_01_02.mat', '20120430_seq2_01_01.mat', '20120430_seq2_01_02.mat', '20120430_seq3_01_01.mat', '20120430_seq3_01_02.mat', '20120430_seq5_01_01.mat', '20120430_seq5_01_02.mat', '20120430_se

In [3]:
# Open the mean annotation folder and sort it to correspond to the features folders

even_names_files = list()
index = 0

for name in name_of_files:
    if(index%2 == 0):
        even_names_files.append(name[:-10])
    index += 1

dic2 = {}
number_of_files = 0
path = r'C:/Users/pinouche/Documents/ImperialData/Annotations/two'

for filename in even_names_files:
    dic2[filename] = sio.loadmat(path+'/'+filename+'/'+'meanAnnotation.mat')
    number_of_files = number_of_files + 1
    
print(number_of_files)

sequences_shape = list()
mean_annotations = list()

for filename in even_names_files:
    mean_annotations.append(dic2[filename]['annotations'])
    sequences_shape.append(dic2[filename]['annotations'].shape[0])

array_annotations = np.concatenate(np.asarray(mean_annotations), axis=0)

20


In [4]:
# Getting the features data into usable matrix format

def dataMatrix(array1, array2):

    person1 = list()
    person2 = list()

    for frame in range(0, array1.shape[2]):
        for sift in range(0,49):
            person1.append(array1[sift,:,frame])
            person2.append(array2[sift,:,frame])

    tmp_arr_mat1 = np.reshape(np.asarray(person1),(array1.shape[2],6272))
    tmp_arr_mat2 = np.reshape(np.asarray(person2),(array2.shape[2],6272))
    frame_vector = np.concatenate([tmp_arr_mat1,tmp_arr_mat2], axis=1)
    
    return(frame_vector)


In [5]:
# Get all the values from the SIFT values for each of the frame from the dictionnary

key=0
concat = []

for val in name_of_files:
    if(key%2 == 0):
        person1 = dic[val]['SIFT']
    else:
        person2 = dic[val]['SIFT']
        concat.append(dataMatrix(person1, dic[val]['SIFT']))
    key +=1
        
Big_list = np.concatenate(concat)
print(Big_list.shape)


(20640, 12544)


In [6]:
# Perform SVD decomposition and keep 500 PCs, which accounts for 3/4 of the total variance

n_components = 500
svd = decomposition.TruncatedSVD(n_components=n_components, algorithm='arpack')
svd.fit(Big_list)
print(svd.explained_variance_ratio_.sum())

# 250 components: 67.3%, 400: 73.7%, 500: 75.4%

KeyboardInterrupt: 

In [ ]:
# Get the new data of dimension  (number of frames * 500)

svd_data_Big_list = np.dot(Big_list,np.transpose(svd.components_))
numpy.savetxt("PCA_Big_list.csv", svd_data_Big_list, delimiter=",")